In [1]:
import os
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from PIL import Image, ImageDraw

import threading

In [2]:
def parse_filename(filename):
    """ ファイル名からx, y, rを抽出する """
    parts = filename.split('_')
    if len(parts) >= 4:
        x, y, r = map(int, parts[1:4])
        return x, y, r
    return None, None, None

def draw_circle_on_image(image_path, x, y, r):
    """ 画像に円を描画する """
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        draw.ellipse((x-r, y-r, x+r, y+r), outline='red', width=2)
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        return img_byte_arr.getvalue()


In [3]:
import os

def display_images_with_circle_and_controls(image_directory, interval=0.5):
    # 画像ファイルのリストを取得
    images = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    image_path = [os.path.join(image_directory, image) for image in images]
    total_images = len(images)

    # 再生・停止・戻る・削除のためのフラグと現在の画像インデックス
    is_playing = False
    current_image = 0

    # 画像とファイル名、インデックスを表示するためのウィジェットを作成
    x, y, r = parse_filename(images[0]) if images else (None, None, None)
    image_with_circle = draw_circle_on_image(image_path[0], x, y, r) if x is not None else open(image_path[0], 'rb').read()
    image_widget = widgets.Image(
        value=image_with_circle,
        format='png',
        width=224,
        height=224,
    )
    filename_label = widgets.Label(value=f"{images[0]} - 1/{total_images}" if images else "No images available")

    # 画像を更新する関数
    def update_image():
        nonlocal current_image
        if images:
            x, y, r = parse_filename(images[current_image])
            image_with_circle = draw_circle_on_image(image_path[current_image], x, y, r) if x is not None else open(image_path[current_image], 'rb').read()
            image_widget.value = image_with_circle
            filename_label.value = f"{images[current_image]} - {current_image + 1}/{total_images}"

    # 自動再生を行う関数
    def autoplay_images():
        nonlocal is_playing, current_image
        while is_playing and current_image < total_images:
            update_image()
            time.sleep(interval)
            current_image += 1
            if current_image >= total_images:
                current_image = 0

    # 再生ボタンのイベントハンドラ
    def on_play_clicked(b):
        nonlocal is_playing
        if not is_playing and images:
            is_playing = True
            threading.Thread(target=autoplay_images).start()

    # 停止ボタンのイベントハンドラ
    def on_stop_clicked(b):
        nonlocal is_playing
        is_playing = False

    # 戻るボタンのイベントハンドラ
    def on_back_clicked(b):
        nonlocal current_image
        if images:
            if current_image > 0:
                current_image -= 1
            else:
                current_image = total_images - 1
            update_image()

    # 削除ボタンのイベントハンドラ
    def on_delete_clicked(b):
        nonlocal current_image, total_images
        if images:
            # 削除処理
            os.remove(image_path[current_image])
            del images[current_image]
            del image_path[current_image]
            total_images -= 1
            if current_image >= total_images:
                current_image = total_images - 1
            update_image()

    # 再生・停止・戻る・削除ボタンの作成
    play_button = widgets.Button(description="Play")
    stop_button = widgets.Button(description="Stop")
    back_button = widgets.Button(description="Back")
    delete_button = widgets.Button(description="Delete")
    play_button.on_click(on_play_clicked)
    stop_button.on_click(on_stop_clicked)
    back_button.on_click(on_back_clicked)
    delete_button.on_click(on_delete_clicked)

    # ウィジェットを表示
    display(widgets.HBox([play_button, stop_button, back_button, delete_button]))
    display(filename_label)
    display(image_widget)

# 使用例
display_images_with_circle_and_controls('../dataset/label/', interval=0.5)

# ノートブック環境での実行のため、実際のディレクトリパスは指定できませんが、
# これをあなたのJupyter Notebookにコピーして、適切なパスで実行してください。


Label(value='165_-257_-173_0_82cc7984c9e947d990ef5887cc64df3c.jpg - 1/200')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xe0\x00\x00\x00\xe0\x08\x02\x00\x00\x00\x95O\xfd…